# 33 MeV D on Ta 

This notebook documents the steps taken to normalize the simulated MCNP foil results.

The notebook has dependecies on PScripts, which can be cloned from:
https://github.com/jamesbevins/PyScripts

Once cloned, the path to the PyScripts/src directory must be added to the user's PYTHONPATH environement variable for this notebook to work.

## MCNP

The experimental configuration was simulated in MCNP to get the expected reaction rates.  The Meulder's deuterons on 197Au was used as the source, and the foil pack was suspended center beam 34.62 cm from the wall.

The results are located at <path>\Dropbox\UCB\Research\ETAs\88Inch\Data\Simulated\Activation\33MevTa\BeamOnly\Model

They are sumarized at:

In [1]:
import os.path

simFoilPath = os.path.abspath('../../../Activation/33MevTa/BeamOnly/SimFoilResults.xlsx')

if os.path.isfile(simFoilPath): 
    print 'The file exists at: {}'.format(simFoilPath)
else:
    print 'ERROR: The file DOES NOT exist at: {}'.format(simFoilPath)

The file exists at: C:\Users\James\Dropbox\UCB\Research\ETAs\88Inch\Data\Simulated\Activation\33MevTa\BeamOnly\SimFoilResults.xlsx


## Experimental Parameters

In [2]:
runTime = 132*60
totCurrent = 3216 * 2E-5 * 1E6 #microA
srcStrength = totCurrent/runTime #microC
print "The MCNP normalization factor is 4.04E9 (from Notebook)"

The MCNP normalization factor is 4.04E9 (from Notebook)


## Read in Simulation data

Read from the designated file path.  Manipulate the file to build a dataframe with the information needed for the count plan.  

In [3]:
import numpy as np
import pandas as pd

simVaultFoils = pd.read_excel(simFoilPath, sheetname='MCNP_Results')
from GeneralNuclear.BasicNuclearCalcs import get_decay_const, production_decay

# Delete unneccesary columns for readability and make the index the reactant
simVaultFoils.index = simVaultFoils.Rx
del simVaultFoils['Foil']
del simVaultFoils['Rx']
del simVaultFoils['AW']
del simVaultFoils['Radius [cm]']
del simVaultFoils['Thickness [cm]']
del simVaultFoils['Density [g/cm^3]']
del simVaultFoils['Density Uncertainty']
del simVaultFoils['Lambda [s^-1]']

# Rename columns for ease of access and add in the statistics column
simVaultFoils.columns = ['product', 'gammaEnergy', 'br', 'normalization', 'rxRate',
                         'rxRateSigma', 'mass', 'weightFrac', 'volume', 'volSigma',
                         'halfLife']

# Put branching ratios in fractional form
simVaultFoils['br'] = simVaultFoils['br']/100.

# Calculate the activity following transit time decay for each rx; delete columns no longer needed
simVaultFoils['initActivity'] = 0.0
simVaultFoils['activityUncert'] = 0.0
simVaultFoils['specActivity'] = 0.0
simVaultFoils['specActUncert'] = 0.0
for ind in simVaultFoils.index:
    simVaultFoils.at[ind, 'initActivity'] = production_decay(simVaultFoils.at[ind, 'halfLife'], 0, runTime, \
                                                    simVaultFoils.at[ind, 'rxRate'], \
                                                    simVaultFoils.at[ind, 'normalization'], \
                                                    simVaultFoils.at[ind, 'volume'], 360)\
                                                    *get_decay_const(simVaultFoils.at[ind,'halfLife'])\
                                                    *simVaultFoils.at[ind,'br']
    simVaultFoils.at[ind, 'activityUncert'] = simVaultFoils.at[ind, 'initActivity']*\
                                              np.sqrt(simVaultFoils.at[ind, 'rxRateSigma']**2+
                                                      (simVaultFoils.at[ind, 'volSigma']/simVaultFoils.at[ind, 'volume'])**2)
    simVaultFoils.at[ind, 'specActivity'] = simVaultFoils.at[ind, 'initActivity'] / simVaultFoils.at[ind, 'mass']
    simVaultFoils.at[ind, 'specActUncert'] = np.sqrt((simVaultFoils.at[ind, 'activityUncert']/simVaultFoils.at[ind, 'initActivity'])**2
                                                     +(0.002/simVaultFoils.at[ind, 'mass'])**2)\
                                             * simVaultFoils.at[ind, 'specActivity']
    
del simVaultFoils['rxRate']
del simVaultFoils['normalization']
#del simVaultFoils['volume']
del simVaultFoils['rxRateSigma']
#del simVaultFoils['volSigma']

print simVaultFoils

            product  gammaEnergy      br    mass  weightFrac    volume  \
Rx                                                                       
27Al(n,p)      Mg27      843.760  0.7180   5.306    1.000000  1.989021   
27Al(n,a)      Na24     1368.630  0.9999   5.306    1.000000  1.989021   
115In(n,n')  In115M      336.241  0.4580  14.326    0.957100  2.016775   
115In(n,g)   In116M     1293.560  0.8480  14.326    0.957100  2.016775   
197Au(n,2n)   Au196      355.700  0.8090   3.298    1.000000  0.179801   
197Au(n,g)    Au198      411.800  0.9562   3.298    1.000000  0.179801   
58Ni(n,2n)     Ni57     1377.630  0.8170  16.934    0.680769  1.924422   
58Ni(n,p)      Co58      810.760  0.9945  16.934    0.680769  1.924422   
90Zr(n,2n)     Zr89      909.150  0.9904  13.189    0.514500  2.026053   

             volSigma    halfLife  initActivity  activityUncert  specActivity  \
Rx                                                                              
27Al(n,p)     0.00711  

## Notes

1) Ni58(n,2n)Ni57 channel will not be clean. There will be contributions from Ni60(n,4n)Ni57 (threshold 33.15 MeV) and Ni61(n,5n)Ni57 (threshold 41.09 MeV) channels. Unfortunatetly, nuclear data does not exist for these channels.

2) Ni58(n,p)Co58 channel will not be clean. There will be contributions from Ni60(n,2np)Co58 (threshold 20.32 MeV), Ni61(n,3np)Co58 (threshold 28.27 MeV), and Ni62(n,4np)Co58 (threshold 39.03 MeV) channels. Unfortunatetly, nuclear data does not exist for these channels.

3) Zr90(n,2n)Zr89 channel will not be clean. There will be contributions from Zr91(n,3n)Zr89 (threshold 19.38 MeV), Zr92(n,4n)Zr89 (threshold 28.80 MeV), and Zr94(n,6n)Zr89 (threshold 43.21 MeV) channels. Unfortunatetly, nuclear data does not exist for these channels. 

For the Ni58(n,p)Co58 and Zr90(n,2n)Zr89 channels, the measured activation rates are likely useless from an unfolding perspective because there are multiple feeding channels due to the energy range covered by the beam.  

The Ni58(n,2n)Ni57 is borderline as Ni60(n,4n)Ni57 (threshold 33.15 MeV) is technically possible, but I think it would be a minor factor.  Since it is such a small overlap for Ni60(n,4n)Ni57, we can address this in the model. 